In [1]:
# To produce messages programmatically and send to the Consumer
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from kafka import KafkaProducer
import json

# Initialize Spark Session with Kafka dependencies
spark = SparkSession.builder \
    .appName("KafkaStructuredStreaming") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.0") \
    .config("spark.sql.streaming.checkpointLocation", "file:///C:/tmp/kafka-checkpoint") \
    .getOrCreate()
 
spark

25/03/16 17:44:58 WARN Utils: Your hostname, Jennie-Kims-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.137 instead (on interface en0)
25/03/16 17:44:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/haenainglay/.ivy2/cache
The jars for the packages stored in: /Users/haenainglay/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83df049a-9e3a-43ce-8d3e-09d15a9f4680;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.0 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.9.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 203ms :: artifacts dl 7ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#ha

In [2]:
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "test") \4
    .option("startingOffsets", "latest") \
    .load()
kafka_df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [3]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

# Create a Kafka producer
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',  # Replace with your Kafka broker
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Serialize JSON data
)

# Sample data (10) to send to the Consumer
id = 1
for i in range (10):
    msg = dt_string + " Hello Kafka Message No. " + str(i) 
    data = {"id": i, "message": msg }
    # Send the JSON message
    producer.send("test", value=data)
    producer.flush()  # Ensure all messages are sent

print("Message sent successfully!")

Message sent successfully!
